In [1]:
%load_ext tensorboard
import datetime
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import numpy as np
import dataset


In [9]:
import importlib
importlib.reload(dataset)
MAXLEN = 80
BATCH_SIZE = 16  # 512
data = dataset.load_file(BATCH_SIZE, 0.1, maxlen=MAXLEN,
                         filenames=['texts/short_table.txt', 'texts/treasure_island.txt'])  # 'texts/bible.txt', 

In [21]:
l2 = tf.keras.regularizers.l2
l1 = tf.keras.regularizers.l1

EMBED_DIM = 2**10  # larger -> quicker opening. knee at 1024

inp = tf.keras.Input(shape=(data.input_texts.shape[1],), batch_size=BATCH_SIZE)

h = layers.Embedding(len(data.letters_table), EMBED_DIM, mask_zero=True)(inp)
# h = layers.Dropout(DROP)(h)

h = layers.Bidirectional(layers.GRU(128, return_sequences=True, recurrent_regularizer=None), merge_mode='sum')(h)  # =l2(1e-6)
# h = layers.Dropout(0.5)(h) 
h1 = layers.Bidirectional(layers.GRU(128, return_sequences=True, recurrent_regularizer=None), merge_mode='sum')(h)
# h1 = layers.Dropout(0.5)(h1) 
h = h1  # layers.Add()([h1, h])
    
h = layers.Dense(128, activation='relu', kernel_regularizer=l2(5e-5))(h)
# h = layers.Dropout(0.5)(h1) 
#h = layers.Add()([h1, h])
# for k in range(1):
#     h = layers.Add()([h, Dense(256)(h)])

h = layers.Dense(data.niqqud_texts.shape[2])(h)
model_niqqud = tf.keras.Model(inputs=[inp], outputs=[h])
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_niqqud.compile(loss='mean_squared_logarithmic_error', optimizer=adam, metrics=['accuracy'])

plot_model(model_niqqud, to_file='model.png')
model_niqqud.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(16, 80)]                0         
_________________________________________________________________
embedding_8 (Embedding)      (16, 80, 1024)            79872     
_________________________________________________________________
bidirectional_15 (Bidirectio (16, 80, 128)             886272    
_________________________________________________________________
bidirectional_16 (Bidirectio (16, 80, 128)             198144    
_________________________________________________________________
dense_12 (Dense)             (16, 80, 128)             16512     
_________________________________________________________________
dense_13 (Dense)             (16, 80, 49)              6321      
Total params: 1,187,121
Trainable params: 1,187,121
Non-trainable params: 0
_________________________________________________

In [ ]:

log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
def fit(EPOCHS):
    return model_niqqud.fit(data.input_texts, data.niqqud_texts,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(data.input_validation, data.niqqud_validation),
          callbacks=[
              # tf.keras.callbacks.ModelCheckpoint(filepath='niqqud_checkpoints/ckpt_{epoch}', save_weights_only=True),
              tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2, verbose=1),
              tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=0, min_lr=0.0001),
              # tensorboard_callback,
          ]
    )

history = fit(EPOCHS=10)

Train on 11880 samples, validate on 1320 samples
Epoch 1/10
11880/11880 [==============================] - 51s 4ms/sample - loss: 0.0034 - accuracy: 0.8018 - val_loss: 0.0020 - val_accuracy: 0.8733
Epoch 2/10
11880/11880 [==============================] - 35s 3ms/sample - loss: 0.0016 - accuracy: 0.8966 - val_loss: 0.0014 - val_accuracy: 0.9118
Epoch 3/10
11880/11880 [==============================] - 35s 3ms/sample - loss: 0.0012 - accuracy: 0.9216 - val_loss: 0.0011 - val_accuracy: 0.9272
Epoch 4/10
  336/11880 [..............................] - ETA: 32s - loss: 0.0010 - accuracy: 0.9361

In [ ]:
history1 = fit(EPOCHS=10)

In [ ]:
import matplotlib.pyplot as plt

for n, v in enumerate(['accuracy', 'loss'], 1):
    plt.subplot(1, 2, n)
    plt.plot(history.history[v][0:])
    plt.plot(history.history['val_' +  v][0:])
    plt.title('Model ' + v)
    plt.ylabel(v)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    
plt.tight_layout()

In [ ]:

model = tf.keras.Model(inputs=[inp], outputs=[tf.keras.layers.Softmax()(h)])

def print_predictions(batch):
    results = data.merge(batch, ns=model.predict(batch))

    for r in results:
        print(r)



print_predictions(data.input_validation[4:4+BATCH_SIZE])

In [ ]:
# model.load_weights(tf.train.latest_checkpoint('niqqud_checkpoints/'))

In [ ]:
print(history.history['val_accuracy'])

In [ ]:
%tensorboard --logdir logs\fit